In [1]:
from langchain_openai import AzureOpenAI,AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
import chromadb
import json
from sentence_transformers import SentenceTransformer, util
import os
import re
import requests
import sys
from num2words import num2words
import pandas as pd
import numpy as np
import tiktoken
#from openai import AzureOpenAI
from datetime import datetime, timedelta
from typing import List
from datetime import datetime, timedelta
from langchain.tools import BaseTool
from openai import AzureOpenAI
import openai
from langchain.agents import initialize_agent, AgentType
#from langchain.llms import AzureOpenAI
from langchain.agents import AgentExecutor,create_openai_tools_agent
from langchain.agents import load_tools
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

c:\Users\Admin\Downloads\Project 4\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2023-05-15",
    api_key=os.getenv('API_KEY'),
    azure_endpoint=os.getenv('AZURE_ENDPOINT')
)

In [11]:
llm = AzureChatOpenAI(model_name="gls-gpt-35-turbo", temperature=0, api_key=os.getenv('API_KEY'), azure_endpoint=os.getenv('AZURE_ENDPOINT'),api_version=os.getenv('API_VERSION'))

Method 1: Using Vectordb


In [5]:
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory='./chromadb/hscodes/'
)

In [8]:
# k is the number of chunks to retrieve
retriever = vectordb.as_retriever(search_type="similarity",search_kwargs={'k': 50})

docs = retriever.invoke("I want to import olives.")

docs

[Document(page_content='HS Code: 0711.20.1800\nDescription: Vegetables provisionally preserved, but unsuitable in that state for immediate consumption: > Olives: > Not pitted: > Green in color, in a saline solution, in containers each holding more than 8 kg, drained weight, certified by the importer to be used for repacking or sale as green olives: > Described in additional U.S. note 5 to this chapter and entered pursuant to its provisions\n', metadata={'hs_code': '0711.20.1800'}),
 Document(page_content='HS Code: 2005.70.2542\nDescription: Other vegetables prepared or preserved otherwise than by vinegar or acetic acid, not frozen, other than products of heading 2006: > Olives: > In a saline solution: > Green in color: > Pitted or stuffed: > Other > In containers each holding 8 kg or less, drained weight: > Whole pitted: > Spanish style of a kind described in statistical note 1 to this chapter\n', metadata={'hs_code': '2005.70.2542'}),
 Document(page_content='HS Code: 2005.70.2550\nDes

In [9]:
len(docs)

50

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Act as an expert in asking follow-up questions one at a time. Whenever I ask you a question, you have to ask me follow-up questions one by one to accurately suggest the specific single HS code in full 10 digits. Use the hierarchy to form questions. Strictly ask questions one by one.

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm,question_answering_prompt)

In [11]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="I want to import olives. What is the HS code?")
        ],
    }
)

'Are the olives fresh or preserved?'

In [15]:
document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="The olives are preserved.")
        ],
    }
)

'Are the olives pitted or stuffed?'

In [16]:
document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="Not Pitted")
        ],
    }
)

'Is the olive green in color?'

In [17]:
document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="Not Green")
        ],
    }
)

'Is it canned?'

Method 2: Using Vectordb1

In [6]:
from langchain_community.docstore.document import Document


In [8]:
text1= "HS Code: 2001.10.0000 - This describes Cucumbers including gherkins prepared or preserved by vinegar or acetic acid."
text2= "HS Code: 2001.90.1000 - This describes Capers prepared or preserved by vinegar or acetic acid in immediate containers holding more than 3.4 kg."
text3= "HS Code: 2001.90.2000 - This describes Capers  prepared or preserved by vinegar or acetic acid in other types of containers."
text4= "HS Code: 2001.90.2500 - This describes Artichokes Vegetables prepared or preserved by vinegar or acetic acid."
text5="HS Code: 2001.90.3000 - This describes Beans Vegetables prepared or preserved by vinegar or acetic acid."
text6="HS Code: 2001.90.3300 - This describes Nopalitos Vegetables prepared or preserved by vinegar or acetic acid."
text7="HS Code: 2001.90.3400 - This describes Onions Vegetables prepared or preserved by vinegar or acetic acid."
text8="HS Code: 2001.90.3500 - This describes Pimientos Vegetables prepared or preserved by vinegar or acetic acid."
text9="HS Code: 2001.90.3800 - This describes other Vegetables excluding Artichokes,Beans,Nopalitos,Onions and Pimientos prepared or preserved by vinegar or acetic acid."
text10="HS Code: 2001.90.4200 - This describes Chestnuts, other than Chinese water chestnuts prepared or preserved by vinegar or acetic acid."
text11="HS Code: 2001.90.4500 - This describes Mangoes fruit prepared or preserved by vinegar or acetic acid."
text12="HS Code: 2001.90.4800 - This describes Chinese water Chestnuts prepared or preserved by vinegar or acetic acid."
text13="HS Code: 2001.90.5000 - This describes Walnuts prepared or preserved by vinegar or acetic acid." 
text14="HS Code: 2001.90.6000 - This describes Vegetables, fruit, nuts and other edible parts of plants, prepared or preserved by vinegar or acetic acid other than cucumbers,Capers,Vegetables,Chestnuts, Mangoes and Walnuts indicating other preserved edible parts not specified elsewhere."
text15="HS Code: 2002.10.0020 - This describes Tomatoes, whole or in pieces in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid."
text16="HS Code: 2002.10.0080 - This describes Tomatoes, whole or in pieces in other types of containers prepared or preserved otherwise than by vinegar or acetic acid."
text17="HS Code: 2002.90.4000 - This describes Tomatoes in the form of powder prepared or preserved otherwise than by vinegar or acetic acid."
text18="HS Code: 2002.90.8010 - This describes Tomatoes paste in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid."
text19="HS Code: 2002.90.8020 - This describes Tomatoes paste in any type of containers prepared or preserved otherwise than by vinegar or acetic acid."
text20="HS Code: 2002.90.8030 - This describes Tomato puree in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid."
text21="HS Code: 2002.90.8040 - This describes Tomato puree in any type of containers prepared or preserved otherwise than by vinegar or acetic acid."
text22="HS Code: 2002.90.8050 - This describes Tomatoes in any other type of form excluding paste,powder,whole or in pieces and puree prepared or preserved otherwise than by vinegar or acetic acid."

In [9]:
doc1 = Document(page_content=text1)
doc2 = Document(page_content=text2)
doc3 = Document(page_content=text3)
doc4 = Document(page_content=text4)
doc5 = Document(page_content=text5)
doc6 = Document(page_content=text6)
doc7 = Document(page_content=text7)
doc8 = Document(page_content=text8)
doc9 = Document(page_content=text9)
doc10 = Document(page_content=text10)
doc11 = Document(page_content=text11)
doc12 = Document(page_content=text12)
doc13 = Document(page_content=text13)
doc14 = Document(page_content=text14)
doc15 = Document(page_content=text15)
doc16 = Document(page_content=text16)
doc17 = Document(page_content=text17)
doc18 = Document(page_content=text18)
doc19 = Document(page_content=text19)
doc20 = Document(page_content=text20)
doc21 = Document(page_content=text21)
doc22 = Document(page_content=text22)


In [10]:
docs=[doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10,doc11,doc12,doc13,doc14,doc15,doc16,doc17,doc18,doc19,doc20,doc21,doc22]

In [11]:
len(docs)

22

In [13]:
vectordb1 = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory='C:\\Users\\Admin\\Downloads\\Project 4\\vdb4'
)

In [13]:
vectordb1 = Chroma(
    embedding_function=embeddings,
    persist_directory='C:\\Users\\Admin\\Downloads\\Project 4\\vdb4'
)

In [113]:
retriever = vectordb1.as_retriever(search_type="similarity",search_kwargs={'k': 22})

docs = retriever.invoke("I want to import olives.")

docs

[Document(page_content='HS Code: 2001.90.3300 - This describes Nopalitos Vegetables prepared or preserved by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.10.0020 - This describes Tomatoes, whole or in pieces in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.8010 - This describes Tomatoes paste in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.10.0080 - This describes Tomatoes, whole or in pieces in other types of containers prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2001.90.6000 - This describes Vegetables, fruit, nuts and other edible parts of plants, prepared or preserved by vinegar or acetic acid other than cucumbers,Capers,Vegetables,Chestnuts, Mangoes and Walnuts indicating other preserved edible parts not specified e

In [117]:
len(docs)

22

In [116]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Act as an expert in asking follow-up questions one at a time. Whenever I ask you a question, you have to ask me follow-up questions one by one to accurately suggest the specific single HS code in full 10 digits.Strictly ask questions one by one.

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm,question_answering_prompt)

In [125]:
from langchain_core.messages import HumanMessage,AIMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="I want to import tomatoes. What is the HS code?"),
            AIMessage(content="Are the tomatoes whole or in pieces?"),
            HumanMessage(content="They are neither whole nor in pieces."),
            AIMessage(content="Are the tomatoes in the form of paste?"),
            HumanMessage(content="No"),
            AIMessage(content="Are the tomatoes in the form of puree?"),
            HumanMessage(content="Yes"),
            AIMessage(content="Are the tomatoes in containers holding less than 1.4 kg?"),
            HumanMessage(content="No"),
            AIMessage(content="Are the tomatoes in any other type of containers?"),
            HumanMessage(content="Yes"),
            AIMessage(content="Are the tomatoes prepared or preserved otherwise than by vinegar or acetic acid?"),
            HumanMessage(content="Yes")
        ],
    }
)

'The specific HS code for the tomatoes you described, in the form of puree, in any other type of containers, and prepared or preserved otherwise than by vinegar or acetic acid, is 2002.90.8050.'

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------------------

In [86]:

from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [87]:
retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="I want to import tomatoes")
        ],
    }
)

{'messages': [HumanMessage(content='I want to import tomatoes')],
 'context': [Document(page_content='HS Code: 2002.10.0020 - This describes Tomatoes, whole or in pieces in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid.'),
  Document(page_content='HS Code: 2002.90.8010 - This describes Tomatoes paste in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid.'),
  Document(page_content='HS Code: 2002.10.0080 - This describes Tomatoes, whole or in pieces in other types of containers prepared or preserved otherwise than by vinegar or acetic acid.'),
  Document(page_content='HS Code: 2002.90.8050 - This describes Tomatoes in any other type of form excluding paste,powder,whole or in pieces and puree prepared or preserved otherwise than by vinegar or acetic acid.'),
  Document(page_content='HS Code: 2002.90.8020 - This describes Tomatoes paste in any type of containers prepared or preserved other

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------

In [65]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [66]:
prompt_search_query = ChatPromptTemplate.from_messages([
MessagesPlaceholder(variable_name="chat_history"),
("user","{input}"),
("user","Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

In [67]:
prompt_get_answer = ChatPromptTemplate.from_messages([
("system", "Act as a expert in asking follow-up questions one at a time.Whenever I ask you a question you have to ask me follow up questions one by one to accurately suggest the specific single HS code in full 10 digits.Strictly ask questions one by one from below context:\\n\\n{context}"),
MessagesPlaceholder(variable_name="chat_history"),
("user","{input}"),
])

In [68]:
retriever = vectordb1.as_retriever(search_type="similarity",search_kwargs={'k': 10})

In [69]:
from langchain.chains import create_history_aware_retriever

In [70]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt_search_query)


In [71]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt_get_answer)

In [72]:
response=retriever_chain.invoke({"input":"I want to import Tomatoes."})


In [73]:
response

[Document(page_content='HS Code: 2002.10.0020 - This describes Tomatoes, whole or in pieces in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.10.0080 - This describes Tomatoes, whole or in pieces in other types of containers prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.8010 - This describes Tomatoes paste in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.8050 - This describes Tomatoes in any other type of form excluding paste,powder,whole or in pieces and puree prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.4000 - This describes Tomatoes in the form of powder prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.8020 - This

In [82]:
# No chat history
chat_history = [HumanMessage(content="I want to import tomatoes"),AIMessage(content='Are you looking to import whole tomatoes or processed tomato products?'),
                HumanMessage(content="processed tomato products"),AIMessage(content='Are you specifically interested in importing tomato paste, tomato puree, or any other specific processed tomato product?'),
                HumanMessage(content="Tomato Puree"),AIMessage(content='Are you looking to import tomato puree in containers holding less than 1.4 kg or in any type of containers?'),
                HumanMessage(content="any other types of containers"),AIMessage(content='Are you looking for tomato puree that is prepared or preserved otherwise than by vinegar or acetic acid?'),
                ]
response = retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Yes"
})

response

[Document(page_content='HS Code: 2002.90.8030 - This describes Tomato puree in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.8040 - This describes Tomato puree in any type of containers prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.8010 - This describes Tomatoes paste in containers holding less than 1.4 kg prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.8020 - This describes Tomatoes paste in any type of containers prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.10.0080 - This describes Tomatoes, whole or in pieces in other types of containers prepared or preserved otherwise than by vinegar or acetic acid.'),
 Document(page_content='HS Code: 2002.90.4000 - This describes Tomatoes in the form of powder prepared or 

In [83]:
chat_history = [HumanMessage(content="I want to import tomatoes"),AIMessage(content='Are you looking to import whole tomatoes or processed tomato products?'),
                HumanMessage(content="processed tomato products"),AIMessage(content='Are you specifically interested in importing tomato paste, tomato puree, or any other specific processed tomato product?'),
                HumanMessage(content="Tomato Puree"),AIMessage(content='Are you looking to import tomato puree in containers holding less than 1.4 kg or in any type of containers?'),
                HumanMessage(content="any other types of containers"),AIMessage(content='Are you looking for tomato puree that is prepared or preserved otherwise than by vinegar or acetic acid?'),
                ]
result=document_chain.invoke({"chat_history":chat_history,"context":response,"input":"Yes"})
result

'Based on your requirements, the specific HS code for importing tomato puree in any type of containers, prepared or preserved otherwise than by vinegar or acetic acid, would be 2002.90.8040.'

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------

In [84]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_search_query = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

prompt_get_answer = ChatPromptTemplate.from_messages([
    ("system", "Act as a expert in asking follow-up questions one at a time.Whenever I ask you a question you have to ask me follow up questions one by one to accurately suggest the specific single HS code in full 10 digits.Strictly ask questions one by one from below context:\\n\\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

retriever = vectordb1.as_retriever(search_type="similarity", search_kwargs={'k': 10})

In [91]:
def update_chat_history(human_message, ai_message):
    chat_history.append(HumanMessage(content=human_message))
    chat_history.append(AIMessage(content=ai_message))
    return chat_history

In [92]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt_search_query)
document_chain = create_stuff_documents_chain(llm, prompt_get_answer)

In [104]:
response = retriever_chain.invoke({"chat_history": chat_history, "input": "yes"})
result = document_chain.invoke({"chat_history": chat_history, "context": response, "input": "yes"})
print(result)

Based on your requirements, the specific HS code for importing tomato puree in any type of containers, prepared or preserved otherwise than by vinegar or acetic acid, is 2002.90.8040.


In [103]:
update_chat_history(human_message="any type of containers",ai_message="Are you looking for tomato puree that is prepared or preserved otherwise than by vinegar or acetic acid?")

[HumanMessage(content='I want to import tomatoes'),
 AIMessage(content='Are you looking to import whole tomatoes or processed tomato products?'),
 HumanMessage(content='processed tomato products'),
 AIMessage(content='Are you specifically interested in importing tomato paste, tomato puree, or tomatoes in other forms such as powder or pieces?'),
 HumanMessage(content='Tomato puree'),
 AIMessage(content='Are you looking to import tomato puree in containers holding less than 1.4 kg or in any type of containers?'),
 HumanMessage(content='any type of containers'),
 AIMessage(content='Are you looking for tomato puree that is prepared or preserved otherwise than by vinegar or acetic acid?')]

In [105]:
#-------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------

In [107]:
from langchain.chains import create_retrieval_chain

retriever_chain = create_history_aware_retriever(llm, retriever, prompt_search_query)
document_chain = create_stuff_documents_chain(llm, prompt_get_answer)

rag_chain = create_retrieval_chain(retriever_chain, document_chain)

In [108]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [113]:
conversational_rag_chain.invoke(
    {"input": "yes"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Based on your responses, the specific HS code for importing tomato puree in any type of containers, prepared or preserved otherwise than by vinegar or acetic acid, is 2002.90.8040.'

In [114]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: I want to import tomatoes

AI: Are you looking to import whole tomatoes or tomatoes in pieces?

User: none of those

AI: Are you looking to import tomato paste, tomato puree, or tomatoes in any other form?

User: tomato puree

AI: Are you looking to import tomato puree in containers holding less than 1.4 kg or in any type of containers?

User: in any type of containers

AI: Are you looking to import tomato puree that is prepared or preserved otherwise than by vinegar or acetic acid?

User: yes

AI: Based on your responses, the specific HS code for importing tomato puree in any type of containers, prepared or preserved otherwise than by vinegar or acetic acid, is 2002.90.8040.



In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------